In [1]:
!pip install pyautogui

     |████████████████████████████████| 59 kB 157 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 132 kB 444 kB/s eta 0:00:01
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.53-py3-none-any.whl size=36583 sha256=165768d9a1305f06f36561eef19940ce58fae0d1adb6c1e85e28e73dc0d2e95c
  Stored in directory: /home/mohamed/.cache/pip/wheels/54/04/f7/55704a9d374ed923785a4fdc0ef00151fa25306b3b93345532
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl size=11078 sha256=08c6a2946272fb0c776e27e9a74c2afd735d94b6351766c9c7b0bd15d12321d3
  Stored in directory: /home/mohamed/.cache/pip/wheels/91/e3/04/184bda98ec7f7d5f0ba1f372bcf7b3ba86325151ae1dd690fe
  Created wheel for pyscreeze: filename=PyScreeze-0.1.28-py3-none-

In [3]:
!pip install mouse

In [6]:
!pip install mediapipe

     |████████████████████████████████| 32.7 MB 1.7 kB/s eta 0:00:01     |██████████▎                     | 10.6 MB 190 kB/s eta 0:01:57     |█████████████████               | 17.3 MB 152 kB/s eta 0:01:42     |█████████████████████████▉      | 26.5 MB 127 kB/s eta 0:00:50     |██████████████████████████▊     | 27.3 MB 299 kB/s eta 0:00:19


In [5]:
!pip install autopy

     |████████████████████████████████| 8.3 MB 21 kB/s  eta 0:00:01     |████████████████▉               | 4.4 MB 1.5 MB/s eta 0:00:03


In [25]:
import cv2
import numpy as np
import sys
import mouse
#sys.path.insert(1, 'F:/machine learning/modules/')
import handLandmarkMP as hlm
import pyautogui
import time
import autopy

In [2]:
screen_width, screen_height= pyautogui.size()

In [3]:
print("width: {}, height: {}".format(screen_width,screen_height))

width: 1366, height: 768


In [4]:
screen_height//3

256

In [5]:
hands = hlm.HandDetector()

In [6]:
np.isclose([11, 9], [0, 0],atol = 10)

array([False,  True])

In [7]:
autopy.mouse.toggle(autopy.mouse.Button.LEFT, down = True)

In [36]:
cap = cv2.VideoCapture(0)
success, img = cap.read()
img_center = (img.shape[1]//2,img.shape[0]//2)
x1,y1 = img_center[0] - (455//2), img_center[1] - (256//2)
x2,y2 = img_center[0] + (455//2), img_center[1] + (256//2)
a,b,c,d = x1,y1,x2,y2
e,f,g,h = 1,1,1366,768

start = time.time()

avg_factor = 7
avg_index = 0
history_x = np.ones(avg_factor) * 1366
history_y = np.ones(avg_factor) * 768
x_before = 1366 // 2
y_before = 768 // 2

left_clicked = False
right_clicked = False
middle_clicked = False

start_point = (x1,y1)
end_point = (x2,y2)
rectangle_color = (155,155,155)
rectangle_thickness = 3
frames = 0

font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    success, img = cap.read()
    #img = cv2.resize(img,(screen_width//2,screen_height//2))
    img = cv2.flip(img, 1)
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue
    img = hands.findHands(img)
    hands_positions = hands.findPositions(img)
    #print(hands_positions)
    #mouse.move(screen_width//2, screen_height//2, absolute=False, duration=0)
    img = cv2.rectangle(img, start_point, end_point, rectangle_color, rectangle_thickness)
    if hands_positions:
        index = np.array(hands_positions['hand1']['index'][-1])
        middle = np.array(hands_positions['hand1']['middle'][-1])
        ring = np.array(hands_positions['hand1']['ring'][-1])
        thumb = np.array(hands_positions['hand1']['thumb'][-1])
        index_middle_knuckle = np.array(hands_positions['hand1']['index'][-2])
        
        #print(f"middle co-ordinates : {middle}")
        #print(f"index co-ordinates : {index}")
        print(f"thumb co-ordinates : {index_middle_knuckle}")
        
        # get the ecludian distance between middle finger tip and index finger tip
        x, y = np.abs(middle-index)
        x_ring, y_ring = np.abs(middle - ring)
        x_thumb_index, y_thumb_index = np.abs(thumb - index_middle_knuckle)
        
        
        dist_middle_index = int((x**2 + y**2)**0.5)
        dist_middle_ring = int((x_ring**2 + y_ring**2)**0.5)
        dist_thumb_index = int((x_thumb_index**2 + y_thumb_index**2)**0.5)
        
        print(f"thumb to index distance {dist_thumb_index}")
        #print(f"middle to index distance {dist_middle_index}")
        #print(f"middle to ring distance {dist_middle_ring}")
        
        #print(f"index pos:{index} middle pos {middle}")
        
        # map x and y values from the frame to the screen size
        x_ = e + (middle[0] - a) * (g - e) / (c - a)
        y_ = f + (middle[1] - b) * (h - f) / (d - b)
        
        if (np.isclose(dist_middle_index,60,atol = 10) and not left_clicked):
            autopy.mouse.click()
            left_clicked = True
        
        if (np.isclose(dist_middle_ring,65,atol = 5) and not right_clicked):
            autopy.mouse.click(autopy.mouse.Button.RIGHT)
            right_clicked = True
        
        if (np.isclose(dist_thumb_index,30,atol = 7) and not (left_clicked or right_clicked):
            autopy.mouse.click(autopy.mouse.Button.MIDDLE)
            middle_clicked = True
            
            
        if(dist_middle_index < 50):
            left_clicked = False
        
        if(dist_middle_ring < 55):
            right_clicked = False
            
        if(dist_thumb_index > 70):
            middle_clicked = False
        
        if (np.all(np.isclose([x_,y_],[x_before,y_before],atol = 10))):
            x_ = x_before
            y_ = y_before
        
        # appending current average
        history_x[avg_index] = int(x_)
        history_y[avg_index] = int(y_)
        
        # averaging over position history
        pos_avg_x = int(np.average(history_x))
        pos_avg_y = int(np.average(history_y))
        
        #print(history_x)
        
        mouse.move(pos_avg_x, pos_avg_y, absolute=True, duration=0)
        #cv2.circle(img, hands_positions['hand1']['middle'][-1], 0, (0, 0, 0), 6)
        
        x_before = x_
        y_before = y_
    
    
    end = time.time()
    avg_index += 1
    
    if (avg_index / avg_factor == 1):
        avg_index = 0
    fps = str(int(frames / (end-start)))
    frames += 1
    cv2.putText(img, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow("Image", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

thumb co-ordinates : [492 293]
thumb to index distance 125
thumb co-ordinates : [476 272]
thumb to index distance 128
thumb co-ordinates : [452 256]
thumb to index distance 127
thumb co-ordinates : [435 244]
thumb to index distance 125
thumb co-ordinates : [428 239]
thumb to index distance 127
thumb co-ordinates : [414 232]
thumb to index distance 122
thumb co-ordinates : [396 213]
thumb to index distance 119
thumb co-ordinates : [392 209]
thumb to index distance 117
thumb co-ordinates : [390 209]
thumb to index distance 118
thumb co-ordinates : [392 214]
thumb to index distance 116
thumb co-ordinates : [396 217]
thumb to index distance 118
thumb co-ordinates : [400 218]
thumb to index distance 117
thumb co-ordinates : [400 219]
thumb to index distance 117
thumb co-ordinates : [400 219]
thumb to index distance 117
thumb co-ordinates : [392 221]
thumb to index distance 117
thumb co-ordinates : [387 221]
thumb to index distance 114
thumb co-ordinates : [383 220]
thumb to index distance 1

thumb co-ordinates : [452 230]
thumb to index distance 57
thumb co-ordinates : [450 226]
thumb to index distance 59
thumb co-ordinates : [449 225]
thumb to index distance 61
thumb co-ordinates : [449 226]
thumb to index distance 62
thumb co-ordinates : [451 221]
thumb to index distance 70
thumb co-ordinates : [448 217]
thumb to index distance 72
thumb co-ordinates : [447 218]
thumb to index distance 69
thumb co-ordinates : [448 218]
thumb to index distance 71
thumb co-ordinates : [449 219]
thumb to index distance 70
thumb co-ordinates : [449 219]
thumb to index distance 71
thumb co-ordinates : [448 222]
thumb to index distance 70
thumb co-ordinates : [448 221]
thumb to index distance 72
thumb co-ordinates : [448 223]
thumb to index distance 71
thumb co-ordinates : [448 223]
thumb to index distance 73
thumb co-ordinates : [448 226]
thumb to index distance 71
thumb co-ordinates : [447 225]
thumb to index distance 73
thumb co-ordinates : [449 226]
thumb to index distance 73
thumb co-ordin

thumb co-ordinates : [490 271]
thumb to index distance 36
thumb co-ordinates : [489 268]
thumb to index distance 64
thumb co-ordinates : [491 266]
thumb to index distance 106
thumb co-ordinates : [493 264]
thumb to index distance 102
thumb co-ordinates : [492 267]
thumb to index distance 101
thumb co-ordinates : [493 266]
thumb to index distance 97
thumb co-ordinates : [494 269]
thumb to index distance 60
thumb co-ordinates : [493 275]
thumb to index distance 40
thumb co-ordinates : [492 275]
thumb to index distance 40
thumb co-ordinates : [492 276]
thumb to index distance 38
thumb co-ordinates : [493 277]
thumb to index distance 34
thumb co-ordinates : [493 276]
thumb to index distance 36
thumb co-ordinates : [493 276]
thumb to index distance 39
thumb co-ordinates : [492 276]
thumb to index distance 39
thumb co-ordinates : [492 276]
thumb to index distance 37
thumb co-ordinates : [491 276]
thumb to index distance 41
thumb co-ordinates : [491 276]
thumb to index distance 39
thumb co-or

thumb co-ordinates : [490 281]
thumb to index distance 79
thumb co-ordinates : [489 281]
thumb to index distance 79
thumb co-ordinates : [490 281]
thumb to index distance 79
thumb co-ordinates : [489 280]
thumb to index distance 78
thumb co-ordinates : [489 281]
thumb to index distance 76
thumb co-ordinates : [488 280]
thumb to index distance 77
thumb co-ordinates : [489 280]
thumb to index distance 78
thumb co-ordinates : [489 279]
thumb to index distance 79
thumb co-ordinates : [489 279]
thumb to index distance 80
thumb co-ordinates : [489 279]
thumb to index distance 78
thumb co-ordinates : [488 279]
thumb to index distance 77
thumb co-ordinates : [487 279]
thumb to index distance 75
thumb co-ordinates : [491 282]
thumb to index distance 76
thumb co-ordinates : [491 283]
thumb to index distance 77
thumb co-ordinates : [491 284]
thumb to index distance 75
thumb co-ordinates : [489 282]
thumb to index distance 77
thumb co-ordinates : [488 281]
thumb to index distance 75
thumb co-ordin

thumb co-ordinates : [181 257]
thumb to index distance 49
thumb co-ordinates : [199 261]
thumb to index distance 46
thumb co-ordinates : [216 263]
thumb to index distance 48
thumb co-ordinates : [256 271]
thumb to index distance 55
thumb co-ordinates : [237 403]
thumb to index distance 51
thumb co-ordinates : [247 398]
thumb to index distance 37
thumb co-ordinates : [259 408]
thumb to index distance 26
thumb co-ordinates : [494 317]
thumb to index distance 126
thumb co-ordinates : [463 294]
thumb to index distance 128
thumb co-ordinates : [447 289]
thumb to index distance 140
thumb co-ordinates : [414 285]
thumb to index distance 139
thumb co-ordinates : [404 282]
thumb to index distance 137
thumb co-ordinates : [380 269]
thumb to index distance 138
thumb co-ordinates : [362 256]
thumb to index distance 135
thumb co-ordinates : [357 256]
thumb to index distance 135
thumb co-ordinates : [348 256]
thumb to index distance 132
thumb co-ordinates : [336 255]
thumb to index distance 128
thum

In [21]:
mouse.move(1366, 640, absolute=True, duration=0.1)

In [29]:
autopy.key.tap(key = autopy.key.Code.PAGE_UP,)